In [9]:
from mlx_lm import load

# Define your model to import
model_name = "mlx-community/Meta-Llama-3-8B-Instruct-4bit"

# Loading model
model, tokenizer = load(model_name)

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 99864.38it/s]


In [10]:
from mlx_lm import generate

In [7]:
# Define the role of the chatbot
chatbot_role = "You are a math professor chatbot. You answer with clear step by step solutions."

# Define a mathematical problem
math_question = "Please compute the integral of f(x) = exp(-x^2) over the x domain of (- infinity, infinity), use LaTeX."

# Set up the chat scenario with roles
messages = [
    {"role": "system", "content": chatbot_role},
    {"role": "user", "content": math_question}
]

# Apply the chat template to format the input for the model
input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

# Decode the tokenized input back to text format to be used as a prompt for the model
prompt = tokenizer.decode(input_ids)

# Generate a response using the model
response = generate(model, tokenizer, max_tokens=512, prompt=prompt)